In [20]:
import os
import gensim
from gensim import models
import pprint
from nltk.stem import WordNetLemmatizer
from pprint import pprint
import glob

In [2]:
from gensim.parsing.preprocessing import STOPWORDS
stopwords = set(STOPWORDS)
my_stopwords = {'approach','paper','article','model','viewpoint','conclusion','discussion','method','introduction','section','framework','based','result','proposed','theory','information','process','analysis','data','problem','review','springer','verlag'}
my_stopwords = set.union(stopwords,my_stopwords)
my_stopwords = frozenset(my_stopwords)

In [24]:
data_folder = '../new_data/'
files = os.listdir(data_folder)
files.remove('.DS_Store') #remove arquivo criado automaticamente sei lá porquê
print(files)

['scopus (8)_cn.ris', 'scopus (9)_us.ris', 'scopus (14)_us.ris', 'scopus (1)_cn.ris', 'scopus (15)_us.ris', 'scopus (8)_us.ris', 'scopus_br.ris', 'scopus (1)_us.ris', 'scopus (9)_cn.ris', 'scopus (2)_cn.ris', 'scopus (3)_us.ris', 'scopus (2)_us.ris', 'scopus (16)_us.ris', 'scopus (3)_cn.ris', 'scopus (6)_cn.ris', 'scopus (13)_us.ris', 'scopus (12)_cn.ris', 'scopus (7)_us.ris', 'scopus (6)_us.ris', 'scopus (2)_br.ris', 'scopus (7)_cn.ris', 'scopus (12)_us.ris', 'scopus (11)_cn.ris', 'scopus (4)_us.ris', 'scopus_cn.ris', 'scopus (5)_cn.ris', 'scopus (10)_us.ris', 'scopus (4)_cn.ris', 'scopus (11)_us.ris', 'scopus (10)_cn.ris', 'scopus (1)_br.ris', 'scopus_us.ris', 'scopus (5)_us.ris']


In [25]:
texts = []
dois = []
author_address = []
publication_year = []
lem = WordNetLemmatizer()

In [26]:
# pensar em tirar duplicatas (comparando pelo título)
for ff in files: 
    f = data_folder + ff
    fid = open(f,'r',encoding='utf8')
    data = readris(fid)

    for i,entry in enumerate(data):
    
        try:
            title = entry['title']
            abstract = entry['abstract']
            
            if 'doi' in entry:
                dois += [entry['doi']]
            else:
                dois += [['']]

            if 'author_address' in entry:
                country = entry['author_address'].split(',')[-1]
                if country[0] == ' ':
                    country = country[1:]
                author_address += [country]
            else:
                author_address += ['-']

            if 'publication_year' in entry:
                publication_year += [entry['publication_year']]
            else:
                publication_year += ['-']

            text = title + ' ' + abstract
            text = gensim.utils.simple_preprocess(text)
            for i,t in enumerate(text):
                text[i] = lem.lemmatize(t)

            text = [word for word in text if word not in my_stopwords]

            texts += [text]

        except:
            pass

    fid.close()


In [27]:
dictionary = gensim.corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in texts]

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=4, workers=4)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.018*"fuzzy" + 0.017*"set" + 0.012*"graph" + 0.011*"rule" + 0.007*"knowledge" + 0.007*"function" + 0.007*"reasoning" + 0.007*"concept" + 0.007*"relation" + 0.006*"structure"
Topic: 1 
Words: 0.016*"material" + 0.016*"machine" + 0.012*"learning" + 0.011*"ml" + 0.010*"structure" + 0.009*"chemical" + 0.008*"design" + 0.008*"property" + 0.007*"artificial" + 0.007*"intelligence"
Topic: 2 
Words: 0.036*"agent" + 0.036*"human" + 0.014*"intelligence" + 0.013*"artificial" + 0.012*"behavior" + 0.010*"ai" + 0.008*"interaction" + 0.008*"autonomous" + 0.007*"environment" + 0.007*"cognitive"
Topic: 3 
Words: 0.025*"wa" + 0.023*"patient" + 0.011*"clinical" + 0.010*"learning" + 0.010*"study" + 0.010*"diagnosis" + 0.009*"accuracy" + 0.008*"performance" + 0.007*"sensitivity" + 0.007*"algorithm"
Topic: 4 
Words: 0.069*"network" + 0.039*"neural" + 0.011*"artificial" + 0.011*"time" + 0.009*"learning" + 0.008*"control" + 0.008*"algorithm" + 0.008*"deep" + 0.006*"performance" + 0.006*"predi

In [28]:
topics = []
scores = []
for ind_doc, bc in enumerate(bow_corpus):
    ind_top, score = sorted(lda_model[bc], key=lambda tup: -1*tup[1])[0]
    topics += [ind_top]
    scores += [score]

In [29]:
ind_doc = 6
topics[ind_doc]
scores[ind_doc]

0.5226966

In [30]:
teste = 450
print('\nTitle & Abs: {}\Country: {}\nDOI:{}\nTopic: {} - Score: {}\n'.format(texts[teste], author_address[teste], dois[teste], topics[teste], scores[teste]))


Title & Abs: ['salient', 'environmental', 'sound', 'detection', 'machine', 'awareness', 'auditory', 'perception', 'essential', 'environment', 'perception', 'saliency', 'detection', 'fundamental', 'basis', 'efficient', 'way', 'achieving', 'task', 'artificial', 'machine', 'intelligent', 'perception', 'sound', 'required', 'provide', 'awareness', 'initiatory', 'step', 'artificial', 'consciousness', 'novel', 'salient', 'environment', 'sound', 'detection', 'machine', 'awareness', 'heterogeneous', 'saliency', 'feature', 'image', 'acoustic', 'channel', 'improve', 'efficiency', 'global', 'informative', 'saliency', 'estimation', 'initially', 'short', 'term', 'shannon', 'entropy', 'series', 'auditory', 'saliency', 'detection', 'presented', 'obtain', 'spectral', 'temporal', 'saliency', 'feature', 'power', 'spectral', 'density', 'mel', 'frequency', 'cepstral', 'coefficient', 'respectively', 'computational', 'bio', 'inspired', 'inhibition', 'return', 'saliency', 'verification', 'improve', 'accuracy

In [31]:
print(len(topics),len(author_address),len(texts),len(dois))

53816 53816 53816 53816


In [ ]:
ind_doc = 430
for index, score in sorted(lda_model[bow_corpus[ind_doc]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic ({}): {}".format(score, index, lda_model.print_topic(index, 10)))


Score: 0.507760226726532	 
Topic (7): 0.044*"image" + 0.021*"recognition" + 0.015*"feature" + 0.012*"object" + 0.008*"proposed" + 0.008*"pattern" + 0.008*"visual" + 0.007*"shape" + 0.007*"algorithm" + 0.007*"motion"

Score: 0.3839522898197174	 
Topic (17): 0.035*"learning" + 0.030*"data" + 0.020*"algorithm" + 0.018*"classification" + 0.018*"feature" + 0.012*"proposed" + 0.010*"set" + 0.010*"machine" + 0.010*"classifier" + 0.010*"method"

Score: 0.03924466669559479	 
Topic (2): 0.028*"rule" + 0.021*"logic" + 0.020*"reasoning" + 0.018*"theory" + 0.016*"fuzzy" + 0.015*"system" + 0.010*"representation" + 0.010*"knowledge" + 0.009*"inference" + 0.009*"model"

Score: 0.03518419712781906	 
Topic (4): 0.055*"network" + 0.017*"detection" + 0.017*"data" + 0.014*"fault" + 0.013*"sensor" + 0.011*"neural" + 0.011*"proposed" + 0.011*"time" + 0.009*"monitoring" + 0.007*"learning"

Score: 0.027981441468000412	 
Topic (3): 0.022*"human" + 0.013*"health" + 0.011*"intelligence" + 0.011*"artificial" + 0.

In [ ]:
len(bow_corpus)

NameError: name 'bow_corpus' is not defined